<a href="https://colab.research.google.com/github/nspiegeln/iml_2022/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IML Task 3 

In [1]:
import torch
import numpy as np

from google.colab import files
uploaded = files.upload()

Saving food.zip to food.zip
Saving test_triplets.txt to test_triplets.txt
Saving train_triplets.txt to train_triplets.txt


In [3]:
import torch
import numpy as np
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as funct
import torch.optim as optim
import math
from matplotlib import image
import tensorflow as tf
from sklearn.model_selection import train_test_split

from google.colab import files
uploaded = files.upload()

Saving test_triplets.txt to test_triplets.txt
Saving train_triplets.txt to train_triplets.txt


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
#importing data
train = np.loadtxt('train_triplets.txt', dtype='str')
test = np.loadtxt('test_triplets.txt', dtype='str')

#unzip folder
!unzip food.zip

# labels ABC - 1, ACB - 0
height = train.shape[0]
y = np.vstack((np.ones((math.floor(height/2), 1)) , np.zeros((math.floor(height/2)+1, 1))))

train = np.hstack((train, y))
# split data set
train_set, val_set = train_test_split(train, test_size=0.1, random_state=42)


In [ ]:
# retrieve image
img_size = 256 
def get_image(image_nr):
    
    image_path = 'food/' + image_nr + '.jpg'
    pre_img = image.imread(image_path)
    
    #rescale image to range[0,1]
    img = tf.image.resize(pre_img, (img_size, img_size))
    img = tf.image.convert_image_dtype(img, tf.float32) / 255.0
    return img


In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

model_loss = nn.TripletMarginLoss(margin=0.5, p=2.0, reduce=None, reduction='mean')

optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.0001)


In [ ]:
def new_network(model):
    
    for param in model.parameters():
        param.requires_grad = False
    
    for param in model.fc.parameters():
        param.requires_grad = True

    for param in model.bn.parameters():
        param.requires_grad = True
    
    for param in model.drop.parameters():
        param.requires_grad = True

    model = model.add_module(Embedding(model))
    return model

In [ ]:
class TripleNet(nn.Module):
    def __init__(self, embedding):
        super(TripleNet, self).__init__()
        self.embedding = embedding
    
    def forward(self, anchor, positive, negative):
        triplet_a = self.embedding(anchor)
        triplet_b = self.embedding(positive)
        triplet_c = self.embedding(negative)
        return triplet_a, triplet_b, triplet_c
        
          
       
class Embedding(nn.Module):
    def __init__(self, resnet):
        super(Embedding, self).__init__()
        
        
        #get last layer
        num_ftrs = resnet.fc.in_features
        self.fc = nn.Linear(num_ftrs, 1000)
        self.bn = nn.BatchNorm1d(1000)
        self.drop = nn.dropout(p = 0)
        
    def forward(self, bio):
        bio = self.drop(funct.relu(self.bn(self.fc(bio))))
        return bio

In [ ]:
if torch.cuda.is_available():
    model = model.cuda()
        
neural_net = TripleNet(new_network(model))

In [ ]:
#training data 
print("Training started")

epochs = 10
is_training = True

for i in range(1, epochs+1):
    training_loss = 0
    val_loss = 0
    
    alpha = 0.5
    if is_training == True:
        for k in range(train_set.size[0]):
            train_a = get_image(train_set[k, 1])
            train_b = get_image(train_set[k, 2])
            train_c = get_image(train_set[k, 3])
            
            if torch.cuda.is_available():
                train_a = train_a.cuda()
                train_b = train_b.cuda()
                train_c = train_c.cuda()
                
            #clear gradients of previous epoch
            optimizer.zero_grad()
            
            #embedd features
            triplet_a, triplet_b, triplet_c = neural_net(train_a, train_b, train_c)
        
            #compute loss
            loss = model_loss(triplet_a, triplet_b, triplet_c)
        
            #compute loss backward
            loss.backward()
            
            #optimize
            optimizer.step()
            
            #update training loss
            training_loss += loss.item()
    training_loss = training_loss/train_set.size[0]
            
    print("Validation started")
    for k in range(val_set.size[0]):
        val_a = get_image(val_set[k, 1])
        val_b = get_image(val_set[k, 2])
        val_c = get_image(val_set[k, 3])
        y_val = val_set[k,4]
        
        if torch.cuda.is_available():
            val_a = val_a.cuda()
            val_b = val_b.cuda()
            val_c = val_c.cuda()
        
        #embedd features
        triplet_a, triplet_b, triplet_c = neural_net(val_a, val_b, val_c)
    
        #compute loss
        loss = model_loss(triplet_a, triplet_b, triplet_c)
    
        val_loss += loss.item()
      
    val_loss = val_loss/val_set.size[0]
    print("epoch:" + i)
    print(training_loss)
    print(val_loss)